<a href="https://colab.research.google.com/github/StevenMena/07MIAR_RedesNeu_DeepLearning/blob/main/07MIAR_VisualizandoCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



Para finalizar con la última práctica del curso vamos a intentar entender un poco más en profundidad que ocurre dentro de una CNN. Para conseguirlo debemos saber que existen dos cosas fundamentales que podemos visualizar:

-  **Los mapas de activaciones a la salida de las capas.** Son simplemente los resultados que obtenemos a la salida de una determinada capa durante el *forward pass*. Normalmente, cuando visualizamos las activaciones de una red con activaciones de tipo ReLU, necesitamos unas cuantas épocas antes de empezar a ver algo útil. Una cosa para la que son muy útiles es para ver si algún filtro está completamente negro para diferentes entradas, es decir, todos sus elementos son siempre 0. Esto significa que el filtro está muerto, y normalmente pasa cuando entrenamos con learning rates altos.

- **Los filtros aprendidos de los bloques convolucionales**. Normalmente, estos filtros son más interpetables en las primeras capas de la red que en las últimas. Sobre todo, es útil visualizar los filtros de la primera, que está mirando directamente a las imágenes de entrada. Una red bien entrenada tendrá filtros perfectamente definidos, al menos en las primeras capas, y sin practicamente ruido. Si por el contrario tuviésemos filtros con mucho ruido podría deberse a que hace falta entrenar más la red, o a que tenemos overfitting y necesitamos algún método de regularización.


A continuación vamos a llevar a cabo un ejemplo para poner todo lo anterior en práctica. **En primer lugar**, vamos a ver cómo se pueden **visualizar las activaciones de la última capa convolucional de nuestra CNN**. Para ello, utilizaremos una técnica llamada Grad-CAM (Gradient Class Activation Map). La idea que hay detrás es bastante sencilla: para averiguar la importancia de una determinada clase en nuestro modelo, simplemente tomamos su gradiente con respecto a la capa convolucional final y luego lo sopesamos con la salida de esta capa.

Este es el esquema de uso de Grad-CAM:

1) Calcular la salida del modelo y la salida de la última capa convolucional para la imagen en la que queremos calcular el mapa de activación (se puede sacar en otras capas convolucionales).

2) Encontrar el índice de la clase que ha predicho el modelo dada la imagen.

3) Calcular el gradiente de la clase predicha con respecto a la última capa convolucional.

4) Promediarlo y luego ponderarlo con la última capa convolucional (multiplicarlos).

5) Normalizar entre 0 y 1 para la visualización.

6) Convertir a RGB y superponerlo a la imagen original.

In [1]:
# Importamos las librerías necesarias
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from vis.utils import utils
from tensorflow.keras import activations

ModuleNotFoundError: ignored

In [ ]:
# Construimos el modelo, en este caso VGG16 con los pesos de imagenet
model = VGG16(weights='imagenet', include_top=True)
# Compilamos el modelo
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
# Visualizamos el modelo
model.summary()

In [ ]:
# Hacemos los imports necesarios para calcular los Grand-Cams
import tensorflow as tf
from tensorflow.python.ops.gradients_impl import image_grad
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow does not work on Google Colab notebooks, which is why we are using cv2_imshow instead


def Grad_CAMs (img, model, layer, INTENSITY):
  # Procesamos la imagen
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  # Calculamos los gradientes de la capa de la cual queremos sacar la activación
  with tf.GradientTape() as tape:
    #Extraemos la capa para la cual estamos interesados en obtener el mapa de activación
    last_conv_layer = model.get_layer(layer)
    # Construimos los modelos que vamos a utilizar (desde inputs hasta outputs y desde inputs hasta la última capa convolucional)
    iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
    model_out, last_conv_layer = iterate(x)
    # Obtenemos la clase predicha apra la muestra dada
    class_predict= np.argmax(model_out[0])
    class_out = model_out[:,class_predict]
    #Calculamos los gradientes
    grads = tape.gradient(class_out, last_conv_layer) 
    #promedio de gradientes
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
  # Construimos el mapa de calor
  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  heatmap = heatmap.reshape((14, 14))
  # Hacemos un resize del mapa de calor al tamaño de la imagen y lo multiplicamos por la imagen
  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
  heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
  image_grad = heatmap * INTENSITY + img
  return heatmap, image_grad, class_predict


Acto seguido **cargamos un par de imágenes** sobre las que vamos a ver los mapas de activaciones.

In [ ]:
# imports necesarios
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6) # tamaño de las imágenes

# Cargamos dos imágenes
img1 = utils.load_img('https://image.ibb.co/ma90yJ/ouzel2.jpg', target_size=(224, 224))
img2 = utils.load_img('https://image.ibb.co/djhyky/ouzel1.jpg', target_size=(224, 224))

# Las mostramos
f, ax = plt.subplots(1, 2)
ax[0].imshow(img1)
ax[0].grid(False)
ax[0].axis('off')
ax[1].imshow(img2)
ax[1].grid(False)
ax[1].axis('off')

A la función `Grad_CAMs` tenemos que pasarle el **modelo**,  **la capa** para la que queremos ver las activaciones, **la imagen** para la que queremos ver las activaciones y **la intensidad** con la que queremos ponderar el mapa de activaciones.
    
En este caso, la clase para que las que queremos ver las activaciones es la clase predicha (aunuqe nosotros también le podríamos introducir el id de la clase que queremos ver las activaciones. Y qué es eso del id de la clase para la que queremos ver las activaciones? Pues que en el caso de la VGG16 con los pesos de la ImageNet, la clase pájaro es la 20, por lo cual, si le metemos una imagen de un pájaro, debería activarse bastante, e indicarnos en qué se fija para decidir que efectivamente es un pájaro. Si le metiésemos un 64, buscaría una green mamba, que es una serpiente por lo que las activaciones deberían ser mucho menores.


![Paj_serp](https://drive.google.com/uc?id=1RcJ2tFw4_lLtwTHK8xUl0QpsMTK1-MF2)

Para conocer el listado completo de las 1000 clases de ImageNet con sus correspondiente IDs haced click en el siguiente enlace: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a

In [ ]:
# Llamamos a la función Grad_CAMs y le pasamos las imágenes correspondientes, el modelo, la capa para la que queremos calcular el mapa de activación y la intensidad
heatmap_1, imagen_1, class_out_1 = Grad_CAMs (img1, model,'block5_conv3', 0.5)
heatmap_2, imagen_2, class_out_2 = Grad_CAMs (img2, model,'block5_conv3', 0.5)

print("Clase predicha para la primera imagen:" + str(class_out_1))
cv2_imshow(imagen_1)
print("Clase predicha para la segunda imagen:" + str(class_out_2))
cv2_imshow(imagen_2)

¿Qué os ha parecido esto?¿En qué **partes del pájaro** está prestando **atención** la **CNN** para tomar una decisión y **clasificar**?



---



Una vez vistos los mapas de activación, vamos a proceder a ver distintos **filtros de diferentes capas**. 



Creamos las funciones necesarias:

In [ ]:
# Creamos algunas funciones que utilizaremos después
import tensorflow as tf
def create_image ():
  return tf.random.uniform((224,224,3), minval=-0.5, maxval=0.5)

def plot_image (image, title= 'random'):
  image = image - tf.math.reduce_min(image)
  image = image / tf.math.reduce_max(image)
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])
  plt.title(title)
  plt.show()

In [ ]:
import tensorflow as tf
# Creamos la función que nos permiti´ra visualizar los filtros de cualquier capa
def visualize_filter (layer_name, f_index= None, iters=50, conv=1):
  #Creamos el submodelo hasta la capa que le hayamos indicado
  submodel= tf.keras.models.Model (model.input, model.get_layer(layer_name).output)
  num_filters=submodel.output.shape[-1]
  if f_index is None:
    f_index= random_randint(0, num_filters -1)
  assert num_filters> f_index, "f_index is out of bounds"
  image= create_image()
  verbose_step = int(iters/10)
  #Calculamos los gradientes
  for i in range(0, iters):
    with tf.GradientTape() as tape:
      tape.watch(image)
      if conv==1:
      # Para capas convolucionales
        out =submodel(tf.expand_dims(image, axis=0))[:,:,:, f_index]
      else:
      #Para capas lineales
        out =submodel(tf.expand_dims(image, axis=0))[:, f_index]
      loss = tf.math.reduce_mean(out)
    grads= tape.gradient (loss, image)
    grads = tf.math.l2_normalize(grads)
    image += grads *10

    #if (i + 1) % verbose_step ==0:
      #print(f' Iteration: {i +1}, Loss: {loss.numpy(): .4f}')
  plot_image(image, f'{layer_name}, {f_index}')


En primer lugar, vamos a ver qué filtros se utilizan para detectar a los pájaros y serpientes anteriores:





In [ ]:
from vis.utils import utils
from tensorflow.keras import activations
# Extraemos la última capa. Podemos cambiar la función de softmax a lineal para realizar una mejor visualización
model.layers[-1].activation = activations.linear
model = utils.apply_modifications(model)
model.summary()

In [ ]:
#Visualizamos los filtros que se utilizan para la clasificación de pájaros
layer_name ='predictions'
visualize_filter(layer_name, f_index=20, iters=500, conv=0)

In [ ]:
# Veamos ahora los patrones que permiten detectar la serpiente (green mamba, índice 64 de ImageNet)
layer_name ='predictions'
visualize_filter(layer_name, f_index=64, iters=500, conv=0)

Ahora, vamos a ver las visualizaciones de algunos de los filtros de la primera capa convolucional:

In [ ]:
layer_name ='block1_conv1'
filtros =[2,4,20]
for i in filtros:
  visualize_filter(layer_name, f_index=i, iters=500, conv=1)

Vamos a ver ahora filtros de diferentes capas:

In [ ]:
import numpy as np
selected_indices = []
for layer_name in ['block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']:
    layer_idx = utils.find_layer_idx(model, layer_name)
    # Seleccionamos aleatoriamente 4 filtros de cada capa
    submodel= tf.keras.models.Model (model.input, model.get_layer(layer_name).output)
    num_filters=submodel.output.shape[-1]
    filters = np.random.permutation(num_filters)[:4]

    # Generamos el mapa de activaciones
    vis_images = []
    for idx in filters:
        visualize_filter(layer_name, f_index=idx, iters=500, conv=1)